<img align="right" src="images/tf.png"/>
<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/logo.png"/>

---

To get started: consult [start](start.ipynb)

---

# Annotate

Text-Fabric is a tool for computing with read only datasets.
How can you manually annotate an existing dataset?

The scenario is: export the portions that must be annotated into a plain text file, accompanied with
location information.

Use an external tool, e.g.
[BRAT](https://brat.nlplab.org) to manually annotate that text.

Read the resulting annotations, combine them with the location information,
and export the result as a new feature or set of features.

These new features can be published anywhere,
see the [share](share.ipynb) tutorial,
and users that want to make use of the new features, can tell Text-Fabric to fetch it from the
published location alongside the main dataset.

From this point on, the new features act as first class citizens in the dataset.

Note how this does not involve modifying existing datasets!

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
from tf.app import use
from tf.convert.recorder import Recorder
from tf.dataset import Versions

**NB:** We used version 0.4 of this data set to export material, annotate the exported material,
and draw in those  annotation as a bunch of new features.
However, in the meanwhile we have newer versions of the missieven data, where different encoding decisions have been applied.

Rather than doing the annotation work again, we want to migrate the annotations from 0.4 to 0.7.
We shall show how.

First we show how me made the annotations in 0.4, and to that end we use a previous version of the data.

In [3]:
# A = use("missieven:v0.4", hoist=globals())
A = use("missieven:v0.4", checkout="clone", hoist=globals())

rate limit is 5000 requests per hour, with 4998 left for this hour
	connecting to online GitHub repo annotation/app-missieven ... connected
	code/__init__.py...downloaded
	code/app.py...downloaded
	code/config.yaml...downloaded
	code/static...directory
		code/static/display.css...downloaded
		code/static/logo.png...downloaded
	OK


   |     0.00s T author               from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.00s T authorFull           from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.03s T col                  from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.00s T day                  from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.01s T emph                 from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.02s T facs                 from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.09s T fnote                from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.01s T folio                from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.00s T month                from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.00s T page                 from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.00s T place                from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.00s T rawdate              from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.03s T ref                  fr

Text-Fabric has support for exporting data together with location information and then importing new data
and turning it into new features based on the location information.

See [Recorder](https://annotation.github.io/text-fabric/tf/convert/recorder.html).

We show the workflow by selecting a letter, exporting the original text material as plain text,
manually annotating it for named entities with [BRAT](https://brat.nlplab.org) and then saving the output
as a new feature `name`.

# Text selection

We choose volume 1 page 6:

In [5]:
p = A.nodeFromSectionStr("1 6")
for ln in L.d(p, otype="line"):
    A.plain(ln, fmt="layout-orig-notes")

Quite a bit of names. Let's leave out the notes.

In [6]:
for ln in L.d(p, otype="line"):
    A.plain(ln, fmt="layout-orig")

# Recording

We'll prepare this portion of text for annotation outside TF.

What needs to happen is, that we produce a text file and that we remember the postions of the relevant
nodes in that text file.

The [Recorder](https://annotation.github.io/text-fabric/tf/convert/recorder.html).
lets you create a string from nodes,
where the positions of the nodes in that string are remembered.
You may add all kinds of material in between the texts of the nodes.

And it is up to you how you represent the nodes.

We can add strings to the recorder, and we can tell nodes to start and to stop.

We add all words in all lines to the recorder, provided the words belong to the original material.

We add line numbers to each line.

In [7]:
# start a recorder
rec = Recorder()

for ln in L.d(p, otype="line"):
    # start a line node
    rec.start(ln)

    # add the line number
    rec.add(f"{F.n.v(ln)}. ")

    for w in L.d(ln, otype="word"):
        trans = F.transo.v(w)
        # if there is nothing in transo, it is not original text
        if not trans:
            continue

        # start a word node
        rec.start(w)

        # add the word and its trailing punctuation
        rec.add(f"{trans}{F.punco.v(w)}")

        # terminate the word node
        rec.end(w)

    # add a newline
    rec.add("\n")

    # terminate the line node
    rec.end(ln)

As a check, let's print the recorded text:

In [8]:
print(rec.text())

1. 
2. 
3. 
4. Op den 12 deser is een jonge slave van een orancaybij nacht comen
5. swemmen aen onse sloep, cloeck van verstant ende prompt in ’t antwoorden,
6. hebben daeromme den 14en deser goet gevonden 14 soldaeten onder ’t commandement
7. van een sergeant met een prau 3 uuren voor daege aen lant te setten aen
8. d’ander sijde om te maeken een bosschaede waertoe medegenomen werde den
9. overloper tot een guide, niettemin hem gebonden houdende die daertoe seer willich
10. was, haer belastende sich niet te openbaeren, dan voor een persoon van qualiteyt
11. gevangen ofte doot te crijgen. Soo is gebeurt, dat daer quamp alleen met een
12. jongen een groot arancay van Nera, broeder van den sabandaer dewelcke sij
13. tref f ten ende ’t hooft ons hier in ’t casteel gebracht, den voorsz. orancaye was door
14. den jongen ende andere seer wel bekent, 14 daegen te voorens in ’t parlementeren
15. met den onsen gesproken, die hem mede geroemt hadde, 2 van onse Hollanders
16. in den moort van den

and the recorded node positions.

In [9]:
for i in range(20, 30):
    print(f"pos {i}: {rec.positions()[i]}")

pos 20: frozenset({1039, 5054871})
pos 21: frozenset({1039, 5054871})
pos 22: frozenset({1040, 5054871})
pos 23: frozenset({1040, 5054871})
pos 24: frozenset({1040, 5054871})
pos 25: frozenset({1041, 5054871})
pos 26: frozenset({1041, 5054871})
pos 27: frozenset({1041, 5054871})
pos 28: frozenset({1041, 5054871})
pos 29: frozenset({1041, 5054871})


This means that the character on position 20 in the plain text string is part of the text of node 1039 and of node 5054871.

With one statement we write the recorded text and the postions to two files:

In [10]:
rec.write("exercises/v01-p0006.txt")

In [11]:
!head -n 10 exercises/v01-p0006.txt

1. 
2. 
3. 
4. Op den 12 deser is een jonge slave van een orancaybij nacht comen
5. swemmen aen onse sloep, cloeck van verstant ende prompt in ’t antwoorden,
6. hebben daeromme den 14en deser goet gevonden 14 soldaeten onder ’t commandement
7. van een sergeant met een prau 3 uuren voor daege aen lant te setten aen
8. d’ander sijde om te maeken een bosschaede waertoe medegenomen werde den
9. overloper tot een guide, niettemin hem gebonden houdende die daertoe seer willich
10. was, haer belastende sich niet te openbaeren, dan voor een persoon van qualiteyt


In [12]:
!head -n 30 exercises/v01-p0006.txt.pos

5054868
5054868
5054868
5054868
5054869
5054869
5054869
5054869
5054870
5054870
5054870
5054870
5054871
5054871
5054871
1038	5054871
1038	5054871
1038	5054871
1039	5054871
1039	5054871
1039	5054871
1039	5054871
1040	5054871
1040	5054871
1040	5054871
1041	5054871
1041	5054871
1041	5054871
1041	5054871
1041	5054871


<img align="right" src="images/brat1.png" width="500"/>

# Annotating

We head over to a local installation of Brat
and annotate our text.

Left you see a quick and dirty manual annotation of some entities that I performed on
the Brat interface, served locally.

We captured the output of this annotation session into the file `v01-p0006.txt.ann`, it has the following contents:

```
T1	Person 675 679	Nera
T2	GPE 1181 1189	Ternnate
#1	AnnotatorNotes T2	Ternate
T3	Person 1203 1223	Coninck van Spagnien
T4	GPE 1215 1223	Spagnien
T5	Organization 1240 1254	Heeren Staeten
T6	Person 1293 1300	Coninck
T7	Person 1406 1413	Coninck
T8	Organization 1457 1471	Heeren Staeten
T9	GPE 1557 1562	Banda
T10	GPE 1653 1662	Engelsche
T11	Person 58 65	orancay
T12	Person 663 670	arancay
T13	Person 697 706	sabandaer
T14	Person 794 802	orancaye
T15	GPE 965 975	Hollanders
T16	Person 1010 1019	Verhoeven
T17	GPE 1154 1161	Ambojna
#2	AnnotatorNotes T17	Amboina
T18	GPE 1305 1310;1311 1322	ditto 24. plaetse
#3	AnnotatorNotes T18	Ternate
*	Alias T11 T14
R1	Geographical_part Arg1:T2 Arg2:T18
```

Now we want to feed back these annotations as TF features on word nodes.
The Recorder cannot anticipate the formats that tools like Brat deliver their results in.
Therefore, it expects the data to be in a straightforward tabular format.

In this case, we must do a small conversion to bring the output annotations
into good shape, namely a tab separated file
with columns `start end feature1 feature2 ...`

Here we choose to expose the identifier (the `T`n values) as feature1
and the kind of entity as feature2.

In case there is a link between two entities, we want to assign
the earliest `T`number to all entities involved.

We also want to preserve the annotator notes.

In [13]:
def brat2tsv(inh, outh):
    outh.write("start\tend\tentityId\tentityKind\tentityComment\n")
    entities = []
    notes = {}
    maps = {}
    for line in inh:
        fields = line.rstrip("\n").split("\t")
        if line.startswith("T"):
            id1 = fields[0]
            (kind, *positions) = fields[1].split()
            (start, end) = (positions[0], positions[-1])
            entities.append([start, end, id1, kind, ""])
        elif line.startswith("#"):
            id1 = fields[1].split()[1]
            notes[id1] = fields[2]
        elif line.startswith("*"):
            (kind, id1, id2) = fields[1].split()
            maps[id2] = id1
        elif line.startswith("R"):
            (id1, id2) = (f[5:] for f in fields[1].split()[1:])
            maps[id2] = id1
    for entity in entities:
        id1 = entity[2]
        if id1 in maps:
            entity[2] = maps[id1]
        if id1 in notes:
            entity[4] = notes[id1]
        line = "\t".join(entity)
        print(line)
        outh.write(f"{line}\n")

    print(maps)


with open("exercises/v01-p0006.txt.ann") as inh:
    with open("exercises/v01-p0006.txt.tsv", "w") as outh:
        brat2tsv(inh, outh)

675	679	T1	Person	
1181	1189	T2	GPE	Ternate
1203	1223	T3	Person	
1215	1223	T4	GPE	
1240	1254	T5	Organization	
1293	1300	T6	Person	
1406	1413	T7	Person	
1457	1471	T8	Organization	
1557	1562	T9	GPE	
1653	1662	T10	GPE	
58	65	T11	Person	
663	670	T12	Person	
697	706	T13	Person	
794	802	T11	Person	
965	975	T15	GPE	
1010	1019	T16	Person	
1154	1161	T17	GPE	Amboina
1305	1322	T2	GPE	Ternate
{'T14': 'T11', 'T18': 'T2'}


Our recorder knows how to do transform this file in feature data.

In [14]:
features = rec.makeFeatures("exercises/v01-p0006.txt.tsv")

Let's see.

In [15]:
for (feat, data) in features.items():
    print(feat)
    print("\t", data)

entityId
	 {1146: 'T1', 5054879: 'T13', 5054889: 'T17', 1255: 'T2', 1259: 'T3', 1260: 'T3', 1261: 'T4', 1264: 'T5', 1265: 'T5', 5054890: 'T2', 1271: 'T6', 1289: 'T7', 5054892: 'T8', 1298: 'T8', 1299: 'T8', 1314: 'T9', 5054894: 'T9', 1330: 'T10', 5054895: 'T10', 1048: 'T11', 1049: 'T11', 5054871: 'T11', 1144: 'T12', 1150: 'T13', 5054880: 'T11', 1167: 'T11', 5054882: 'T15', 1196: 'T15', 5054883: 'T16', 1203: 'T16', 1250: 'T17', 1273: 'T2', 5054891: 'T2', 1274: 'T2'}
entityKind
	 {1146: 'Person', 5054879: 'Person', 5054889: 'GPE', 1255: 'GPE', 1259: 'Person', 1260: 'Person', 1261: 'GPE', 1264: 'Organization', 1265: 'Organization', 5054890: 'GPE', 1271: 'Person', 1289: 'Person', 5054892: 'Organization', 1298: 'Organization', 1299: 'Organization', 1314: 'GPE', 5054894: 'GPE', 1330: 'GPE', 5054895: 'GPE', 1048: 'Person', 1049: 'Person', 5054871: 'Person', 1144: 'Person', 1150: 'Person', 5054880: 'Person', 1167: 'Person', 5054882: 'GPE', 1196: 'GPE', 5054883: 'Person', 1203: 'Person', 1250: '

We can show this prettier:

In [16]:
for (feat, data) in features.items():
    print(feat)
    for (node, value) in data.items():
        print(f"\t{F.otype.v(node)} {node} => {value}")

entityId
	word 1146 => T1
	line 5054879 => T13
	line 5054889 => T17
	word 1255 => T2
	word 1259 => T3
	word 1260 => T3
	word 1261 => T4
	word 1264 => T5
	word 1265 => T5
	line 5054890 => T2
	word 1271 => T6
	word 1289 => T7
	line 5054892 => T8
	word 1298 => T8
	word 1299 => T8
	word 1314 => T9
	line 5054894 => T9
	word 1330 => T10
	line 5054895 => T10
	word 1048 => T11
	word 1049 => T11
	line 5054871 => T11
	word 1144 => T12
	word 1150 => T13
	line 5054880 => T11
	word 1167 => T11
	line 5054882 => T15
	word 1196 => T15
	line 5054883 => T16
	word 1203 => T16
	word 1250 => T17
	word 1273 => T2
	line 5054891 => T2
	word 1274 => T2
entityKind
	word 1146 => Person
	line 5054879 => Person
	line 5054889 => GPE
	word 1255 => GPE
	word 1259 => Person
	word 1260 => Person
	word 1261 => GPE
	word 1264 => Organization
	word 1265 => Organization
	line 5054890 => GPE
	word 1271 => Person
	word 1289 => Person
	line 5054892 => Organization
	word 1298 => Organization
	word 1299 => Organization
	word 13

Note that we assign entity features to line nodes as well.

If that is undesired, we should not have instructed the Recorder to `rec.add(ln)` above.

# Saving data

The [documentation](https://annotation.github.io/text-fabric/tf/core/fabric.html#tf.core.fabric.FabricCore.save)
explains how to save this data into text-fabric data files.

We choose a location where to save it, the `exercises` directory next to this notebook.

In [18]:
GITHUB = os.path.expanduser("~/github")
ORG = "annotation"
REPO = "tutorials"
PATH = "missieven/exercises"
VERSION = A.version

Note the version: we have built the version against a specific version of the data:

In [19]:
A.version

0.4

Later on, we pass this version on, so that users of our data will get the shared data in exactly the same version as their core data.

We have to specify a bit of metadata for this feature:

In [20]:
metaData = {
    "entityId": dict(
        valueType="str",
        description="identifier of a named entity",
        creator="Dirk Roorda",
    ),
    "entityKind": dict(
        valueType="str",
        description="kind of a named entity",
        creator="Dirk Roorda",
    ),
    "entityComment": dict(
        valueType="str",
        description="comment to a named entity",
        creator="Dirk Roorda",
    ),
}

Now we can give the save command:

In [21]:
location = f"{GITHUB}/{ORG}/{REPO}/{PATH}/entities/tf"
TF.save(nodeFeatures=features, metaData=metaData, location=location, module=VERSION)

  0.00s Exporting 3 node and 0 edge and 0 config features to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.4:
   |     0.00s T entityComment        to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.4
   |     0.00s T entityId             to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.4
   |     0.00s T entityKind           to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.4
  0.01s Exported 3 node features and 0 edge features and 0 config features to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.4


True

# Migrating

We now migrate these annotations to version 0.7, which is different from 0.4 in that the footnote texts have been drawn into
the main text.

We use the mapping from 0.4 nodes to 0.7 nodes, which is available as an edge feature `omap#0.4-0.7` in the
0.7 version of the dataset.

We load both the old and new versions of the dataset.

In [22]:
entityModule = "annotation/tutorials/missieven/exercises/entities/tf"
va = 0.4
vb = 0.7

A = {}

In [23]:
# A[va] = use(f"missieven:v{va}", mod=f"{entityModule}:{va}")
A[va] = use(f"missieven:v{va}", mod=f"{entityModule}:clone", checkout="clone")

rate limit is 5000 requests per hour, with 4983 left for this hour
	connecting to online GitHub repo annotation/app-missieven ... connected


   |     0.00s T entityComment        from ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.4
   |     0.00s T entityId             from ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.4
   |     0.00s T entityKind           from ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.4


In [24]:
# A[vb] = use("missieven", mod="")
A[vb] = use("missieven:clone", checkout="clone")

   |       16s T omap#0.4-0.7         from ~/github/Dans-labs/clariah-gm/tf/0.7


Now we can use the function
[migrateFeatures](https://annotation.github.io/text-fabric/tf/dataset/nodemaps.html#tf.dataset.nodemaps.Versions.migrateFeatures)
from TF to migrate our features.
See also
[nodeMaps](https://annotation.github.io/text-fabric/tf/dataset/nodemaps.html).

In [26]:
V = Versions({va: A[va].api, vb: A[vb].api}, va, vb)

In [27]:
features = ("entityComment", "entityId", "entityKind")

V.migrateFeatures(features, location=location)

  0.00s Exporting 3 node and 0 edge and 0 config features to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.7:
   |     0.00s T entityComment        to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.7
   |     0.00s T entityId             to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.7
   |     0.00s T entityKind           to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.7
  0.01s Exported 3 node features and 0 edge features and 0 config features to ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.7


We load the migrated features:

In [28]:
# A[vb] = use(f"missieven:v{vb}", mod=f"{entityModule}:{vb}")
A[vb] = use("missieven:clone", mod=f"{entityModule}:clone", checkout="clone")

   |     0.00s T entityComment        from ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.7
   |     0.00s T entityId             from ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.7
   |     0.00s T entityKind           from ~/github/annotation/tutorials/missieven/exercises/entities/tf/0.7


We compare the features in both versions

In [43]:
def showFeature(v, f):
    F = A[v].api.F
    Fs = A[v].api.Fs
    T = A[v].api.T

    print(f"Feature {f}")
    for (n, val) in Fs(f).items():
        ntp = F.otype.v(n)
        print(f"{v} {f} ({ntp:<4} {n:>8}) {val:<8} <= {T.text(n)}")

In [44]:
for f in features:
    showFeature(va, f)
    showFeature(vb, f)
    print("")

Feature entityComment
0.4 entityComment (word     1250) Amboina  <= Ambojna 
0.4 entityComment (word     1255) Ternate  <= Ternnate 
0.4 entityComment (word     1273) Ternate  <= ditto
0.4 entityComment (word     1274) Ternate  <= plaetse 
0.4 entityComment (line  5054889) Amboina  <= Ambojna ende van daer naer Ternnate ende soo den Coninck van Spagnien tusschen
0.4 entityComment (line  5054890) Ternate  <= de Heeren Staeten den treves geobserveert werde, metten Coninck van ditto
0.4 entityComment (line  5054891) Ternate  <= plaetse te contracteeren om met sijn hulpe dese plaetse te ocuperen ende hem daer
Feature entityComment
0.7 entityComment (word     1576) Amboina  <= Ambojna 
0.7 entityComment (word     1581) Ternate  <= Ternnate 
0.7 entityComment (word     1599) Ternate  <= ditto
0.7 entityComment (word     1600) Ternate  <= plaetse 
0.7 entityComment (line  5340898) Amboina  <= Ambojna ende van daer naer Ternnate ende soo den Coninck van Spagnien tusschen
0.7 entityComment (lin

# Sharing

In [share](share.ipynb) we show how we can share and reuse these features.

---

# Contents

* **[start](start.ipynb)** start computing with this corpus
* **[search](search.ipynb)** turbo charge your hand-coding with search templates
* **[compute](compute.ipynb)** sink down a level and compute it yourself
* **[exportExcel](exportExcel)** make tailor-made spreadsheets out of your results
* **annotate** export text, annotate with BRAT, import annotations
* **[share](share.ipynb)** draw in other people's data and let them use yours
* **[volumes](volumes.ipynb)** work with selected volumes only

CC-BY Dirk Roorda